# Pre-processing of GLACIOCLIM MB data:

Does the pre-processing of the point MB measurements from GLACIOCLIM (French Alps).

## Setting up:

In [ ]:
import os, sys
sys.path.append(os.path.join(os.getcwd(), '../../')) # Add root of repo to import MBM

import pandas as pd
import os
import warnings
import massbalancemachine as mbm
import geopandas as gpd
import matplotlib.pyplot as plt
import glob
from cmcrameri import cm
from oggm import utils

from regions.French_Alps.scripts.glacioclim_preprocess import *
from regions.French_Alps.scripts.config_FR import *
from regions.Switzerland.scripts.oggm import initialize_oggm_glacier_directories, export_oggm_grids

from regions.Switzerland.scripts.glamos import merge_pmb_with_oggm_data, rename_stakes_by_elevation, check_point_ids_contain_glacier, remove_close_points, check_multiple_rgi_ids


warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

cfg = mbm.FranceConfig()
mbm.utils.seed_all(cfg.seed)
mbm.utils.free_up_cuda()
mbm.plots.use_mbm_style()

## Load data into dictionary of dataframes

Unzip GLACIOCLIM Files

In [ ]:
read_in = False
if read_in == True:
    extract_glacioclim_files(cfg.dataPath + path_PMB_GLACIOCLIM_raw)

### Read all csv from "unzipped" into single dictionary of dataframes

In [ ]:
stakes_csv_all = {}
for path, _, files in os.walk(cfg.dataPath + path_PMB_GLACIOCLIM_raw):
    # Find all .csv files in the current directory
    csv_files = glob.glob(os.path.join(path, '*.csv'))

    # Read each CSV file and add to dictionary
    for file in csv_files:
        try:
            # Use the filename as the key
            key = os.path.splitext(os.path.basename(file))[0]
            # Read the CSV file
            stakes_csv_all[key] = pd.read_csv(file)
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")

print(f"\nTotal number of files processed: {len(stakes_csv_all)}")
display(stakes_csv_all)

### Special case Sarennes. 

The File is not in a typical csv structure, so cant just be read in. For this code to work, in each sheet in the B_SARENNES_94_20.xls file, make a table with start_date, spring_date, end_date, X, Y, Z, winter mb, summer mb, annual mb (requires xlrd package to read in)

In [ ]:
sarennes_path = cfg.dataPath + 'GLACIOCLIM/point/Glacier de SARENNES_discontinued/B_SARENNES_94-20.xls'

# Read all sheets into a dictionary of dfs
all_sheets = pd.read_excel(sarennes_path, sheet_name=None)
sarennes_dfs = extract_sarennes_data(all_sheets)
display(sarennes_dfs)

### Special case Glacier Blanc

In [ ]:
blanc_path = cfg.dataPath + 'GLACIOCLIM/point/Glacier BLANC_unusable/GlacierBlanc2000_2024_pointMB_mwe.xls'

blanc_data = pd.read_excel(blanc_path, skiprows=2).iloc[:, 1:]

blanc_df = extract_blanc_data(blanc_data)

display(blanc_df)

## Convert from Lambert3 / Lambert2 cooridnates to WGS84

In [ ]:
## For some reason there is a 2 in front of the y_lambert2e coordinates in certain years, hardcode remove them:
stakes_csv_all['mdg_Tacul_winter_smb_abl_2007'][
    'y_lambert2e'] = stakes_csv_all['mdg_Tacul_winter_smb_abl_2007'][
        'y_lambert2e'].apply(lambda x: x - 2000000 if x > 2000000 else x)
stakes_csv_all['mdg_Talefre_annual_smb_abl_2006'][
    'y_lambert2e'] = stakes_csv_all['mdg_Talefre_annual_smb_abl_2006'][
        'y_lambert2e'].apply(lambda x: x - 2000000 if x > 2000000 else x)

stakes_csv_all = lambert_transform(stakes_csv_all)
sarennes_dfs = lambert_transform(sarennes_dfs)
blanc_df = lamberttoWGS84(blanc_df)

### Plot stakes over map for visual conformation of coordinate transformation
This needs additional folium package to be installed

In [ ]:
import folium

glacier_dfs = {
    'GEBROULAZ': (stakes_csv_all['geb_annual_smb_abl_1979'], 'red'),
    'ARGENTIERE': (stakes_csv_all['Argentiere_annual_smb_accu_1995'], 'blue'),
    'SAINT_SORLIN': (stakes_csv_all['stso_annual_smb_abl_1957'], 'green'),
    'MER_DE_GLACE':
    (stakes_csv_all['mdg_Leschaux_winter_smb_abl_2020'], 'purple'),
    'SARENNES': (sarennes_dfs['sarennes_complete_winter_2006'], 'orange'),
}

center_lat, center_lon = 45.8736, 6.8770

m = folium.Map(location=[center_lat, center_lon], zoom_start=10)

# Add markers for each glacier
for glacier_name, (df, color) in glacier_dfs.items():
    fg = folium.FeatureGroup(name=glacier_name)

    for _, row in df.iterrows():
        if glacier_name == 'SARENNES':
            stake_id = row['POINT_ID'].split('_')[-1]
            altitude = row['POINT_ELEVATION']
        else:
            stake_id = row['stake_number']
            altitude = row['altitude']

        # Add circle marker with popup
        folium.CircleMarker(
            location=[row['lat'], row['lon']],
            radius=5,
            color=color,
            fill=True,
            fill_color=color,
            popup=f"{glacier_name} - Stake {stake_id}: {altitude}m").add_to(fg)

    fg.add_to(m)

# Add a legend
legend_html = '''
<div style="position: fixed; bottom: 50px; left: 50px; z-index: 1000; background-color: white; padding: 10px; border-radius: 5px;">
    <p><strong>Glaciers</strong></p>
    <p><span style="color: red;">●</span> GEBROULAZ</p>
    <p><span style="color: blue;">●</span> ARGENTIERE</p>
    <p><span style="color: green;">●</span> SAINT SORLIN</p>
    <p><span style="color: purple;">●</span> MER DE GLACE</p>
    <p><span style="color: orange;">●</span> SARENNES</p>
</div>
'''
m.get_root().html.add_child(folium.Element(legend_html))
m

## WGMS Mapping, appending SARENNES and conversion into single dataframe

In [ ]:
stakes_csv_WGMS = {}

# Process each DataFrame in the original dictionary to WGMS format
for key, df in stakes_csv_all.items():
    try:
        required_cols = {
            'lat', 'lon', 'altitude', 'stake_number', 'day_start',
            'month_start', 'year_start', 'day_end', 'month_end', 'year_end'
        }
        if all(col in df.columns for col in required_cols):
            stakes_csv_WGMS[key] = transform_WGMS_df(df, key)
    except Exception as e:
        print(f"Error processing {key}: {str(e)}")

# Append SARENNES to the dictionary
columns = [
    'POINT_ID', 'POINT_LAT', 'POINT_LON', 'POINT_ELEVATION', 'FROM_DATE',
    'TO_DATE', 'POINT_BALANCE', 'GLACIER', 'PERIOD', 'GLACIER_ZONE'
]
for key, df in sarennes_dfs.items():
    stakes_csv_WGMS[key] = df.rename(columns={
        'lat': 'POINT_LAT',
        'lon': 'POINT_LON'
    })[columns]

# Rename Glacier Blanc
blanc_df = blanc_df.rename(columns={'lat': 'POINT_LAT', 'lon': 'POINT_LON'})

Combine dictionary of dfs into 1 df

In [ ]:
# Convert dictionary to single DataFrame
df_pmb_all_raw = pd.concat(stakes_csv_WGMS.values(), ignore_index=True)
# Append Glacier Blanc
df_pmb_all_raw = pd.concat([df_pmb_all_raw, blanc_df], ignore_index=True)
# Sort by date and glacier
df_pmb_all_raw = df_pmb_all_raw.sort_values(
    ['GLACIER', 'GLACIER_ZONE', 'PERIOD', 'FROM_DATE'])
# Add YEAR column to df
df_pmb_all_raw['YEAR'] = df_pmb_all_raw['TO_DATE'].astype(str).str[:4].astype(
    int)
# Add data modification column to keep track of mannual changes
df_pmb_all_raw['DATA_MODIFICATION'] = ''
# Add remark that glacier blanc dates were modified
df_pmb_all_raw.loc[df_pmb_all_raw['GLACIER'] == 'glacier_blanc',
                   'DATA_MODIFICATION'] = 'Hydraulic year dates'
df_pmb_all_raw.head(3)

## Merge stakes that are close

In [ ]:
df_pmb_all_raw_dropped = pd.DataFrame()
for gl in tqdm(df_pmb_all_raw.GLACIER.unique(), desc='Merging stakes'):
    print(f'-- {gl.capitalize()}:')
    df_gl = df_pmb_all_raw[df_pmb_all_raw.GLACIER == gl]
    df_gl_cleaned = remove_close_points(df_gl)
    df_pmb_all_raw_dropped = pd.concat([df_pmb_all_raw_dropped, df_gl_cleaned])
df_pmb_all_raw_dropped.drop(['x', 'y'], axis=1, inplace=True)
df_pmb_all_raw_dropped.reset_index(inplace=True, drop=True)

##### 5. General data cleaning

In [ ]:
print("Rows with NaN values:")
display(df_pmb_all_raw_dropped[df_pmb_all_raw_dropped.isna().any(axis=1)])

print("Rows with zero values:")
display(df_pmb_all_raw_dropped[df_pmb_all_raw_dropped.eq(0).any(axis=1)])

print("Rows with extreme POINT_BALANCE values (>5 or <-15):")
display(
    df_pmb_all_raw_dropped[(df_pmb_all_raw_dropped['POINT_BALANCE'] > 5) |
                           (df_pmb_all_raw_dropped['POINT_BALANCE'] < -15)])

# Saint-Sorlin has POINT_ELEVATION 0.0 on 8 stakes, also about 20 stakes have point balance 0.00, remove them all
mask_zeros = df_pmb_all_raw_dropped.eq(0).any(axis=1)
df_pmb_all_raw_dropped = df_pmb_all_raw_dropped[~mask_zeros]

# stso_summer_smb_abl_2017_ablation_setup2015_14 -88 pmb, nonsensical value, remove it
mask = df_pmb_all_raw_dropped[
    'POINT_ID'] != 'stso_summer_smb_abl_2017_ablation_setup2015_14'
df_pmb_all_raw_dropped = df_pmb_all_raw_dropped[mask]

In [ ]:
annual_inconsistent, winter_inconsistent, summer_inconsistent = check_period_consistency(
    df_pmb_all_raw_dropped)

display(annual_inconsistent)
display(summer_inconsistent)

# 7 short summers but nothing majorly inconsistent, leaving them in
# Annuual, mdg_langue_annual_smb_abl_2008_setup2008_9 is a faulty measurement, goes from 2009 to 2008, pop it
mask = df_pmb_all_raw_dropped[
    'POINT_ID'] != 'mdg_langue_annual_smb_abl_2008_Langue_setup2008_9'
df_pmb_all_raw_dropped = df_pmb_all_raw_dropped[mask]
df_pmb_all_raw_dropped.reset_index(inplace=True, drop=True)

In [ ]:
print('Number of winter and annual samples:', len(df_pmb_all_raw_dropped))
print('Number of annual samples:',
      len(df_pmb_all_raw_dropped[df_pmb_all_raw_dropped.PERIOD == 'annual']))
print('Number of winter samples:',
      len(df_pmb_all_raw_dropped[df_pmb_all_raw_dropped.PERIOD == 'winter']))

# Number of measurements per year:
fig, axs = plt.subplots(2, 1, figsize=(20, 15))
ax = axs.flatten()[0]
df_pmb_all_raw_dropped.groupby(['YEAR', 'PERIOD']).size().unstack().plot(
    kind='bar',
    stacked=True,
    color=[mbm.plots.COLOR_ANNUAL, mbm.plots.COLOR_WINTER],
    ax=ax)
ax.set_title('Number of measurements per year for all glaciers')

ax = axs.flatten()[1]
num_gl = df_pmb_all_raw_dropped.groupby(['GLACIER']).size().sort_values()
num_gl.plot(kind='bar', ax=ax)
ax.set_title('Number of total measurements per glacier since 1951')
plt.tight_layout()

#### Add RGIId and OGGM Data

In [ ]:
# initialize OGGM glacier directories
gdirs, rgidf = initialize_oggm_glacier_directories(
    cfg,
    rgi_region="11",
    rgi_version="62",
    base_url=
    "https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L1-L2_files/2025.6/elev_bands_w_data/",
    log_level='WARNING',
    task_list=None,
)
export_oggm_grids = False
if export_oggm_grids:
    export_oggm_grids(cfg, gdirs)

In [ ]:
# Load glacier outlines
rgi_file = utils.get_rgi_region_file(region="11", version="62")
glacier_outline = gpd.read_file(rgi_file)

# Add RGI IDs through intersection
df_pmb = mbm.data_processing.utils.get_rgi(data=df_pmb_all_raw_dropped,
                                           glacier_outlines=glacier_outline)

print('Number of measurements without RGI:',
      len(df_pmb[df_pmb['RGIId'].isna()]))

# Remove stakes without RGIId, as they wont have OGGM data anyways
print('Drop stakes without RGIs...')
df_pmb = df_pmb.dropna(subset=['RGIId'])

# Create a dictionary mapping from RGIId to glacier name
rgi_to_name_dict = dict(zip(rgidf.RGIId, rgidf.Name))
df_pmb['GLACIER'] = df_pmb['RGIId'].map(rgi_to_name_dict)

rgiids6 = df_pmb[['GLACIER', 'RGIId']].drop_duplicates()
if check_multiple_rgi_ids(rgiids6):
    print(
        "-- Alert: The following glaciers have more than one RGIId. Cleaning up."
    )
else:
    print("-- All glaciers are correctly associated with a single RGIId.")
    df_pmb_clean = df_pmb

df_pmb_clean.reset_index(inplace=True, drop=True)

# Add OGGM data:
df_pmb_topo = merge_pmb_with_oggm_data(df_pmb=df_pmb_clean,
                                       gdirs=gdirs,
                                       rgi_region="11",
                                       rgi_version="62",
                                       verbose=True)
# Restrict to within glacier shape and drop the column
df_pmb_topo = df_pmb_topo[df_pmb_topo['within_glacier_shape'] == True]
df_pmb_topo = df_pmb_topo.drop(columns=['within_glacier_shape'])

# Drop points if aspect or slope is NaN
df_pmb_topo = df_pmb_topo[~df_pmb_topo.aspect.isna()]
df_pmb_topo = df_pmb_topo[~df_pmb_topo.slope.isna()]
df_pmb_topo = df_pmb_topo[~df_pmb_topo.topo.isna()]

df_pmb_topo.reset_index(inplace=True, drop=True)
df_pmb_topo.head()

In [ ]:
print('Number of winter and annual samples:', len(df_pmb_topo))
print('Number of annual samples:',
      len(df_pmb_topo[df_pmb_topo.PERIOD == 'annual']))
print('Number of winter samples:',
      len(df_pmb_topo[df_pmb_topo.PERIOD == 'winter']))
# Unique glaciers, sorted
glacier_list = sorted(df_pmb_topo.GLACIER.unique())
print(f"Number of glaciers: {len(glacier_list)}")
print(f"Glaciers: {glacier_list}")

In [ ]:
# Number of measurements per year:
fig, axs = plt.subplots(2, 1, figsize=(20, 15))
ax = axs.flatten()[0]
df_pmb_topo.groupby(['YEAR', 'PERIOD']).size().unstack().plot(
    kind='bar',
    stacked=True,
    color=[mbm.plots.COLOR_ANNUAL, mbm.plots.COLOR_WINTER],
    ax=ax)
ax.set_title('Number of measurements per year for all glaciers')

ax = axs.flatten()[1]
num_gl = df_pmb_topo.groupby(['GLACIER']).size().sort_values()
num_gl.plot(kind='bar', ax=ax)
ax.set_title('Number of total measurements per glacier since 1951')
plt.tight_layout()

In [ ]:
# Save the combined DataFrame to a CSV file
print(
    f'Saving intermediate output df_pmb_topo.csv to {path_PMB_GLACIOCLIM_csv}')
df_pmb_topo.to_csv(cfg.dataPath + path_PMB_GLACIOCLIM_csv + 'df_pmb_topo.csv',
                   index=False)
display(df_pmb_topo.head(2))

## Check for wrong elevations:

In [ ]:
df_checked, df_bad = flag_elevation_mismatch(df_pmb_topo, threshold=400)

## Add skyview factor:

In [ ]:
# Example of one svf file
rgi_id = df_pmb_topo.loc[0].RGIId

# read ds with svf
path_masked_xr = os.path.join(cfg.dataPath,
                              'GLAMOS/topo/RGI_v6_11/xr_masked_grids')

xr.open_zarr(path_masked_xr + f'/{rgi_id}.zarr').svf.plot()

In [ ]:
path_masked_xr = os.path.join(cfg.dataPath,
                              "GLAMOS/topo/RGI_v6_11/xr_masked_grids")

df_pmb_topo_svf = add_svf_from_rgi_zarr(
    df_pmb_topo,
    path_masked_xr,
    rgi_col="RGIId",
    lon_col="POINT_LON",
    lat_col="POINT_LAT",
    svf_var="svf",
    out_col="svf",
)
df_missing = df_pmb_topo_svf[df_pmb_topo_svf["svf"].isna()].copy()
print("Missing SVF points:", len(df_missing))
print("Glaciers affected:", sorted(df_missing["RGIId"].unique()))

# drop rows where svf is NaN
df_pmb_topo_svf_clean = df_pmb_topo_svf[~df_pmb_topo_svf.svf.isna()]

# Print how many rows were dropped
print(
    f"Dropped {len(df_pmb_topo) - len(df_pmb_topo_svf_clean)} rows due to missing SVF data."
)
df_pmb_topo_svf_clean.reset_index(inplace=True, drop=True)

In [ ]:
plot_missing_svf_for_all_glaciers(
    df_with_svf=df_pmb_topo_svf,
    path_masked_xr=path_masked_xr,
    plot_valid_points=True,
    save_dir=
    None  # or e.g. os.path.join(cfg.dataPath, "diagnostics/svf_missing")
)

In [ ]:
df_pmb_topo_svf_new = add_svf_nearest_valid(
    df_pmb_topo,
    path_masked_xr,
    rgi_col="RGIId",
    lon_col="POINT_LON",
    lat_col="POINT_LAT",
    svf_var="svf",
    out_col="svf",
    max_radius=30,  # ~30 grid cells search; adjust if needed
)

print("Missing SVF points after nearest-valid fill:",
      df_pmb_topo_svf_new["svf"].isna().sum())

plot_missing_svf_for_all_glaciers(
    df_with_svf=df_pmb_topo_svf_new,
    path_masked_xr=path_masked_xr,
    plot_valid_points=True,
    save_dir=
    None  # or e.g. os.path.join(cfg.dataPath, "diagnostics/svf_missing")
)

In [ ]:
plot_glacier_svf_with_points(
    df_with_svf=df_pmb_topo_svf_new,
    path_masked_xr=path_masked_xr,
    rgi_col="RGIId",
    lon_col="POINT_LON",
    lat_col="POINT_LAT",
    svf_col="svf",
    svf_var="svf",
    save_dir=
    None,  # or e.g. os.path.join(cfg.dataPath, "diagnostics/svf_point_plots")
)

## Give new stake IDs:

In [ ]:
df_pmb_new_ids = rename_stakes_by_elevation(df_pmb_topo_svf_new)

# Check the condition
check_point_ids_contain_glacier(df_pmb_new_ids)

print('Number of winter and annual samples:', len(df_pmb_new_ids))
print('Number of annual samples:',
      len(df_pmb_new_ids[df_pmb_new_ids.PERIOD == 'annual']))
print('Number of winter samples:',
      len(df_pmb_new_ids[df_pmb_new_ids.PERIOD == 'winter']))

# Histogram of mass balance
df_pmb_new_ids['POINT_BALANCE'].hist(bins=20)
plt.xlabel('Mass balance [m w.e.]')

## Final cleaning:

In [ ]:
df_pmb_clean = df_pmb_new_ids.copy()
df_pmb_clean['MONTH_START'] = [
    str(date)[4:6] for date in df_pmb_clean.FROM_DATE
]
df_pmb_clean['MONTH_END'] = [str(date)[4:6] for date in df_pmb_clean.TO_DATE]

winter_pmb = df_pmb_clean[df_pmb_clean.PERIOD == "winter"]
annual_pmb = df_pmb_clean[df_pmb_clean.PERIOD == "annual"]

print("Winter measurement months:")
print("  Unique start months:", sorted(winter_pmb["MONTH_START"].unique()))
print("  Unique end months:  ", sorted(winter_pmb["MONTH_END"].unique()))

print("\nAnnual measurement months:")
print("  Unique start months:", sorted(annual_pmb["MONTH_START"].unique()))
print("  Unique end months:  ", sorted(annual_pmb["MONTH_END"].unique()))

# correct mistakes in winter mb. If month end is 06 and mb is negative
# correct to "annual"

# Correct mislabeled winter MB:
# If MONTH_END == 6 and MB is negative, it should be annual (not winter)
mask_fix = ((df_pmb_clean["PERIOD"] == "winter") &
            (df_pmb_clean["MONTH_END"] == '06') &
            (df_pmb_clean["POINT_BALANCE"] < 0))

print("Rows to relabel winter -> annual:", mask_fix.sum())

df_pmb_clean.loc[mask_fix, "PERIOD"] = "annual"

# For now, don't remove anything:
# Save to csv:
df_pmb_clean.to_csv(os.path.join(cfg.dataPath, path_PMB_GLACIOCLIM_csv,
                                 'FR_wgms_dataset_all.csv'),
                    index=False)